# Урок 6. Многоклассовая классификация.

Посмотрим на примере алгоритма логистической регрессии и метода опорных векторов, как работать с различными методами многоклассовой классификации.

### 1.
Вспомните датасет Wine. Загрузите его, разделите на тренировочную и тестовую выборки (random_state=17), используя только [9, 11, 12] признаки.

In [6]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

In [18]:
data = load_wine()
X = data.data[:, [9, 11, 12]]
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=17)



**Задайте тип кросс-валидации с помощью StratifiedKFold: 5-кратная, random_state=17.**

In [19]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

In [28]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

### 2.
Обучите логистическую регрессию (LogisticRegression) с параметром C по умолчанию и random_state=17. Укажите гиперпараметр multi_class='ovr' - по умолчанию многие классификаторы используют именно его. С помощью cross_val_score сделайте кросс-валидацию (используйте объект skf) и выведите среднюю долю правильных ответов на ней (используйте функцию mean). Отдельно выведите долю правильных ответов на тестовой выборке.

In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [47]:
logreg_model = LogisticRegression(random_state=17, multi_class='ovr')
logreg_model.fit(X_train, y_train)
print('Средняя доля верных ответов при 5-ти блочной кросс-валидации: ',
      cross_val_score(logreg_model, X_train, y_train, cv=skf).mean(),
      '\nДоля верных ответов на тестовой выборке: ',
      accuracy_score(y_test, logreg_model.predict(X_test)))

Средняя доля верных ответов при 5-ти блочной кросс-валидации:  0.9096866096866097 
Доля верных ответов на тестовой выборке:  0.9111111111111111


### 3.
Обучите метод опорных векторов (SVC) с random_state=17 и остальными параметрами по умолчанию. Этот метод при мультиклассовой классификации также использует метод "ovr". Сделайте кросс-валидацию (используйте skf) и, как и в предыдущем пункте, выведите среднюю долю правильных ответов на ней. Отдельно выведите долю правильных ответов на тестовой выборке.

In [42]:
from sklearn.svm import SVC

In [67]:
svm_model = SVC(random_state=17)
svm_model.fit(X_train, y_train)
print('Средняя доля верных ответов при 5-ти блочной кросс-валидации: ',
      cross_val_score(svm_model, X_train, y_train, cv=skf).mean(),
      '\nДоля верных ответов на тестовой выборке: ',
      accuracy_score(y_test, svm_model.predict(X_test)))

Средняя доля верных ответов при 5-ти блочной кросс-валидации:  0.6923076923076923 
Доля верных ответов на тестовой выборке:  0.6222222222222222


Как видно из полученной метрики, на тестовой выборке метод с гиперпараметрами по умолчанию работает явно намного хуже логистической регрессии. В целом, SVM достаточно плохо масштабируется на размер обучающего набора данных (как видно, даже с тремя признаками он работает не очень хорошо), но благодаря возможности выбора различных ядер (функций близости, которые помогают разделять данные) и другим гиперпараметрам SVM можно достаточно точно настроить под определенный вид данных. Подробнее на этом останавливаться в контексте данного урока не будем.

### 4.
Для предсказаний обеих моделей постройте матрицу ошибок (confusion matrix) и напишите, какие классы каждая из моделей путает больше всего между собой.

In [46]:
from sklearn.metrics import classification_report, confusion_matrix

In [53]:
confusion_matrix(y_test, logreg_model.predict(X_test))

array([[ 9,  0,  0],
       [ 0, 19,  0],
       [ 0,  4, 13]], dtype=int64)

In [68]:
confusion_matrix(y_test, svm_model.predict(X_test))

array([[ 9,  0,  0],
       [ 0, 19,  0],
       [ 2, 15,  0]], dtype=int64)

Модель логистической регрессии путает второй класс с первым, а метод опорных векторов путает второй класс с первым и вторым

### 5.
Для каждой модели выведите classification report.

In [56]:
print(classification_report(y_test, logreg_model.predict(X_test)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       0.83      1.00      0.90        19
           2       1.00      0.76      0.87        17

    accuracy                           0.91        45
   macro avg       0.94      0.92      0.92        45
weighted avg       0.93      0.91      0.91        45



In [75]:
print(classification_report(y_test, svm_model.predict(X_test), zero_division=0))

              precision    recall  f1-score   support

           0       0.82      1.00      0.90         9
           1       0.56      1.00      0.72        19
           2       0.00      0.00      0.00        17

    accuracy                           0.62        45
   macro avg       0.46      0.67      0.54        45
weighted avg       0.40      0.62      0.48        45

